<a href="https://colab.research.google.com/github/s-jainr/deep-learning-sp23/blob/main/Copy_of_07_AIT_CNN_transfer_learning_Keras_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Copyright
<pre>
You may use and modify this code for research and development purpuses.
Using this code for educational purposes (self-paced or instructor led) without the permission of the author is prohibited.

Paper about Inception V3 model: 
https://arxiv.org/abs/1512.00567

Sources used during preparation of this notebook:
https://keras.io/applications/
https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975
https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb

Copyright (c) 2023 Bálint Gyires-Tóth - All Rights Reserved
</pre>

# Transfer learning

Transfer learning means that a trained network is trained further with additional data and possibly a custom goal.

## Downloading and loading the data

We will use the dogs and cats dataset from https://www.kaggle.com/c/dogs-vs-cats. You can download a fraction of the dataset from https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip.

In [1]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O /tmp/cats_and_dogs_filtered.zip

/tmp/cats_and_dogs_filtered.zip: No such file or directory


In [2]:
!apt-get install unp > /dev/null

The system cannot find the path specified.


In [3]:
!unp /tmp/cats_and_dogs_filtered.zip > /dev/null

The system cannot find the path specified.


Next, we define the names of the directories for training and validation data:


In [4]:
import os
base_dir = 'cats_and_dogs_filtered' 
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Let's list some files from the directories:


In [5]:
train_cat_fnames = os.listdir(train_cats_dir)
print(train_cat_fnames[:10])
train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()
print(train_dog_fnames[:10])

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'cats_and_dogs_filtered\\train\\cats'

Let's investigate some example photos from the dataset:

In [ ]:
train_cat_fnames[:10]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

nrows = 4
ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

next_cat_pix = [os.path.join(train_cats_dir, fname) for fname in train_cat_fnames[:int(ncols*nrows/2)]]
next_dog_pix = [os.path.join(train_dogs_dir, fname) for fname in train_dog_fnames[:int(ncols*nrows/2)]]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

## Loading a pretrained network and training it further (transfer learning)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
img_height=299
img_width=299

In [ ]:
# loading the pretrained Inception V3, without fully-connected layers
base_model = InceptionV3(weights='imagenet', include_top=False)
# Global Average Pooling is a pooling layer, that reshapes the data into 2D, which can be used 
# as input for further dense layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Adding a dense layer with ReLU
x = Dense(1024, activation='relu')(x)
# and finally the output layer for binary classification with a single output and sigmoid activation
predictions = Dense(1, activation='sigmoid')(x)
# defining the model
model = Model(inputs=base_model.input, outputs=predictions)

We train the network in two steps. First, we train the upper dense layers, and while the convolutional layers are 'freezed':

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

# compiling the model after freezing the convolutional layers
model.compile(optimizer='adam', metrics=['accuracy'],loss='binary_crossentropy')

In [ ]:
# using generators to load the images
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')
validation_generator = valid_datagen.flow_from_directory(validation_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')


In [ ]:
# this function trains the model, while using the datagenerators
# please note, that data augmentation is also performed, while training the network
model.fit(train_generator,steps_per_epoch=200,validation_data=validation_generator,validation_steps=10,epochs=3)

In the following part we will take the 2nd step: we will unfreeze the upper convolutional layers of Inception V3.

In [ ]:
print("The convolutional part of Inception V3:")
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

## Exercise

Freeze everything below layer conv2d_56 (including conv2d_56): 

In [ ]:
for layer in base_model.layers[:173]:
       layer.trainable = False
for layer in base_model.layers[173:]:
       layer.trainable = True

Recompiling the model, this time with SGD optimizer, small learning rate (0.0001) and with a momentum (0.9). 

In [ ]:
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), metrics=['accuracy'], loss='binary_crossentropy')

And train the model further, as we did before, and inspect the loss and accuracy:

In [ ]:
model.fit_generator(train_generator,steps_per_epoch=200,validation_data=validation_generator,validation_steps=10,epochs=3)

In [9]:
model.history

NameError: name 'model' is not defined